In [ ]:
/afm03/Q2/Q2051

In [1]:
!which python

/scratch/smp/uqsmac12/.conda/env/stimage/bin/python


In [28]:
import torch
from torch.utils.data import DataLoader, random_split
from torch.nn import functional as F

from torch import optim, nn, utils, Tensor
from torchvision.datasets import MNIST
from torchvision import datasets, transforms
# from torchvision.transforms import ToTensor
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

/scratch/smp/uqsmac12/.conda/env/lit_torch_gp/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
DIR_TILES = '/scratch/smp/uqsmac12/dataset_breast_cancer_9visium'
DIR_ANNDATA_PROCESSED = '/scratch/smp/uqsmac12/dataset_breast_cancer_9visium'

# DIR_PROCESSED_DATA = '/afm03/Q2/Q2051/STimage_project/STimage_dataset/PROCESSED/dataset_breast_cancer_9visium'

In [7]:
file_processed_alex_data = 'all_adata.h5ad'

In [18]:
adata_all = read_h5ad(os.path.join(DIR_PROCESSED_DATA, file_processed_alex_data))

In [19]:
# update metadata for annadata such that it maps to the correct location
adata_all.obs["tile_path"] = adata_all.obs.tile_path.map(
    lambda x: x.replace("/clusterdata/uqxtan9/Xiao/breast_cancer_9visium",
                        DIR_TILES))

In [12]:
gene_list = ["COX6C","TTLL12", "PABPC1", "GNAS", "HSP90AB1", "TFF3", "ATP1A1", "B2M", "FASN", "SPARC", "CD74", "CD63", "CD24", "CD81"]

In [27]:
np.exp(adata_all[0:10,'COX6C'].to_df().values)

array([[ 8.      ],
       [12.      ],
       [15.000002],
       [17.      ],
       [12.      ],
       [10.000001],
       [15.000002],
       [ 7.      ],
       [ 8.      ],
       [ 6.      ]], dtype=float32)

'/scratch/smp/uqsmac12/dataset_breast_cancer_9visium'

In [11]:
adata[:, gene_list

Index(['SAMD11', 'NOC2L', 'KLHL17', 'PLEKHN1', 'PERM1', 'HES4', 'ISG15',
       'AGRN', 'RNF223', 'C1orf159',
       ...
       'F8A1', 'FUNDC2', 'CMC4', 'MTCP1', 'VBP1', 'RAB39B', 'CLIC2', 'TMLHE',
       'SPRY3', 'VAMP7'],
      dtype='object', length=14664)

In [ ]:
class DataGenerator(torch.utils.data.Dataset):
    """
    data generator for multiple branches gene prediction model
    """

    def __init__(self, adata, dim=(299, 299), n_channels=3, genes=None, aug=False, tile_path="tile_path"):
        'Initialization'
        self.dim = dim
        self.adata = adata
        self.n_channels = n_channels
        self.genes = genes
        self.num_genes = len(genes)
        self.aug = aug
        self.tile_path = tile_path
        self.indexes = np.arange(self.adata.n_obs)
        
    def __len__(self):
        'Denotes the number of samples'
        return int(self.adata.n_obs)

    def __getitem__(self, index):
        'Generates one sample of data'
        # Find obs name
        obs_temp = self.adata.obs_names[index]

        # Generate data
        X_img, y = self._load_data(obs_temp)

        return torch.Tensor(X_img), torch.Tensor(y)

    def _load_data(self, obs):
        img_path = self.adata.obs.loc[obs, 'tile_path']
        X_img = Image.open(img_path).convert('RGB')
        X_img = transforms.Resize(self.dim)(X_img)
        X_img = np.array(X_img).astype('uint8')
        #         X_img = np.expand_dims(X_img, axis=0)
        #         n_rotate = np.random.randint(0, 4)
        #         X_img = np.rot90(X_img, k=n_rotate, axes=(1, 2))
        if self.aug:
            X_img = seq_aug(image=X_img)
        y = self._load_label(obs)
        return X_img, y

    def _load_label(self, obs):
        batch_adata = self.adata[obs, self.genes].copy()

        return tuple([batch_adata.to_df()[i].values for i in self.genes])

    def get_classes(self):
        return self.adata.to_df().loc[:, self.genes]